# Imports, Initialization, and Hyperparameters

In [31]:
import pandas as pd
import torch
import numpy as np
import math
import scipy.linalg
import torch_geometric

In [32]:
#Import PPI Data and construct protein list
data = pd.read_csv(r"C:\Users\colef\OneDrive - University of Miami\Documents\College\Research\Wuchty Lab\Data\HI-union.tsv",sep='\t')
data.drop(data[(data['Protein1'] == data['Protein2'])].index,inplace=True)
data.reset_index(drop=True,inplace=True)
protein_set = set([*data.Protein1,*data.Protein2])
protein_list = list(protein_set)

In [33]:
#Construct Edge List
edge_list = torch.tensor([data['Protein1'].apply(lambda x: protein_list.index(x)).values,data['Protein2'].apply(lambda x: protein_list.index(x)).values],dtype=torch.long)
edge_list = torch_geometric.utils.to_undirected(edge_list)

#Contruct Adjacency Matrix from Edge List
def construct_adjacency_matrix(edge_list,protein_list):
    A = torch.zeros(len(protein_list),len(protein_list))
    for i in range(edge_list.shape[1]):
        A[edge_list[0,i],edge_list[1,i]] = 1
        A[edge_list[1,i],edge_list[0,i]] = 1
    return A
A = construct_adjacency_matrix(edge_list,protein_list)

#Construct Degree Matrix from Adjacency Matrix
def construct_degree_matrix(A):
    D = torch.zeros(len(A),len(A))
    for i in range(len(A)):
        D[i,i] = A[i].sum()
    return D
D = construct_degree_matrix(A)

#Define Max Degree
max_degree = D.max().item()

In [34]:
##Hyperparameters

#K: Maximum hop distance (K = 2)
K = 2

#Gamma: Hop discount factor (γ = 0.01)
gamma = 0.01

#Eta: Node degree threshold (η = 15)
eta = 15

#Phi: Compression ratio (φ = 0.2)
# phi = 0.6383333333333333 #Rao
phi = 0.8236799568965517 #Lieberman

#m: Hidden layer size (m = log2(max_degree))
m = int(math.log2(max_degree)) + 1

#p: Embedding layer size (p = 2m)
p = 2*m

# Feature Extraction

In [35]:
features = torch.zeros(len(protein_list),m,dtype=torch.float32)
k = 1

while k <= K:
    for protein in range(len(protein_list)):
        #Get k-hop neighbors of protein
        neighbors = torch_geometric.utils.k_hop_subgraph(protein,k,edge_list)[0]
        
        #Get temporary features using neighbor degrees
        temporary_features = torch.zeros([m])
        for neighbor in neighbors:
            idx = int(math.log2(D[neighbor,neighbor].item()))
            temporary_features[idx] += 1
        
        #Apply hop discount factor and add to features
        temporary_features = pow(gamma,k-1)*temporary_features
        features[protein] += temporary_features
    k += 1  

# Embedding Learning

In [36]:
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        
        #GCN Layers
        self.conv1 = GCNConv(m, m, cached=True)
        nn.init.uniform_(self.conv1.lin.weight,-math.sqrt(6)/math.sqrt(m+m),math.sqrt(6)/math.sqrt(m+m)) #Initialize weights
        self.conv2 = GCNConv(m, p, cached=True)
        nn.init.uniform_(self.conv2.lin.weight,-math.sqrt(6)/math.sqrt(p+m),math.sqrt(6)/math.sqrt(p+m)) #Initialize weights
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.tanh(x)
        x = self.conv2(x, edge_index)
        x = F.tanh(x)
        return x

In [37]:
#Put model and data on the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
features.to(device)

#Use model to get embeddings
embeddings = model(features,edge_list)

# Guiding List

In [38]:
#Reduce size of protein list based on degree threshold η and store features of reduced proteins
reduced_protein_list = []
reduced_features = []
for i in range(len(protein_list)):
    if D[i,i] >= eta:
        reduced_protein_list.append(protein_list[i])
        reduced_features.append(embeddings[i].cpu().detach().numpy())

In [39]:
#Initialize list of norms
norms = []
for i in range(len(reduced_features)):
    norms.append(np.linalg.norm(reduced_features[i]))

#Sort protein list based on norms
sorted_protein_list = [x for _,x in sorted(zip(norms,reduced_protein_list),reverse=True)]

# Graph Compression

In [40]:
#Get neighbors using adjacency matrix
def get_Neighbors(protein_idx,A):
    neighbors = []
    for i in range(len(A)):
        if A[protein_idx,i] == 1:
            neighbors.append(i)
    return neighbors

In [41]:
#Initialize compress_rate and make deep copies of protein list
compress_rate = 0
compressed_A = A.clone()
compressed_D = D.clone()
compressed_edge_list = edge_list.clone()
compressed_protein_list = protein_list.copy()
guided_list = sorted_protein_list.copy()

#Compress loop
while compress_rate < phi:
    
    #Iterate through guided list:
    for protein in guided_list:
        
        #Get minimum degree of neighbors
        protein_idx = compressed_protein_list.index(protein)
        neighbors = get_Neighbors(protein_idx,compressed_A)
        min_degree = max_degree
        for neighbor_idx in neighbors:
            neighbor_degree = compressed_D[neighbor_idx,neighbor_idx].item()
            min_degree = min(min_degree,neighbor_degree)
        
        #Make list of nodes for compression based on minimum degree
        compress_list = []
        for neighbor_idx in neighbors:
            neighbor_degree = compressed_D[neighbor_idx,neighbor_idx].item()
            if neighbor_degree == min_degree:
                compress_list.append(neighbor_idx)
        compress_list.append(protein_idx)
        
        #Compress nodes in compress list
        if len(compress_list) > 1:
            supernode = '+'.join([compressed_protein_list[i] for i in compress_list])
            
            supernode_neighbors = set()
            
            #Store supernode neighbors
            for subnode_idx in compress_list:
                supernode_neighbors = supernode_neighbors | set(get_Neighbors(subnode_idx,compressed_A))
            
            ##Update Adjacency Matrix
            
            #Add Supernode and connections
            compressed_A = torch.cat([compressed_A,torch.zeros(1,len(compressed_A))])
            compressed_A = torch.cat([compressed_A,torch.zeros(len(compressed_A),1)],dim=1)
            
            for neighbor in supernode_neighbors:
                compressed_A[-1,neighbor] = 1
                compressed_A[neighbor,-1] = 1
                
            #Remove subnodes
            for subnode_idx in sorted(compress_list,reverse=True):
                compressed_A = torch.cat([compressed_A[:subnode_idx],compressed_A[subnode_idx+1:]])
                compressed_A = torch.cat([compressed_A[:,:subnode_idx],compressed_A[:,subnode_idx+1:]],dim=1)
            
            ##Update Degree Matrix
            compressed_D = construct_degree_matrix(compressed_A)
            
            ##Update Guided List
            for subnode_idx in compress_list:
                if compressed_protein_list[subnode_idx] in guided_list:
                    guided_list.remove(compressed_protein_list[subnode_idx])
            guided_list.append(supernode)
            
            ##Update Protein List
            compressed_protein_list.append(supernode)
            for subnode_idx in sorted(compress_list,reverse=True):
                compressed_protein_list.remove(compressed_protein_list[subnode_idx])
            

        compress_rate = 1 - len(compressed_protein_list)/len(protein_list)
        print(compress_rate)
        if compress_rate >= phi:
            break

0.011258278145695355
0.01236203090507726
0.013465783664459163
0.015011037527593807
0.01611479028697571
0.016887417218543033
0.018432671081677676
0.022626931567328867
0.0233995584988963
0.024172185430463622
0.02549668874172184
0.02726269315673291
0.028145695364238388
0.028807947019867552
0.029028697571743978
0.029470198675496717
0.029580573951434874
0.030022075055187614
0.03024282560706404
0.030353200883002196
0.030905077262693204
0.032119205298013265
0.0331125827814569
0.03322295805739517
0.034216335540838805
0.034657836644591655
0.035099337748344395
0.03532008830022071
0.03565121412803529
0.03576158940397356
0.035871964679911716
0.035982339955849874
0.03609271523178803
0.037196467991169935
0.03774834437086094
0.037969094922737257
0.03818984547461368
0.03841059602649011
0.03962472406181017
0.041059602649006655
0.04139072847682124
0.041501103752759394
0.04161147902869755
0.04172185430463571
0.04238410596026487
0.04249448123620314
0.04293598233995588
0.04304635761589404
0.043487858719646

In [42]:
#Save compressed protein list
np.savetxt(r"C:\Users\colef\OneDrive - University of Miami\Documents\College\Research\Wuchty Lab\Data\HI-union_compressedLieberman.csv",compressed_protein_list,fmt='%s')